In [95]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font


# Read each tab into separate dataframes from an .xlsm file
xlsm_file = 'DPR_GL_YTD Jun-23.xlsm'
tabs = ['fy22', 'fy21', 'ytd jun-23']

dataframes = []
with pd.ExcelFile(xlsm_file) as xls:
    for tab in tabs:
        df = pd.read_excel(xls, tab)
        dataframes.append(df)

# Append the dataframes together
appended_df = pd.concat(dataframes, ignore_index=True)

# Filter out rows without dates
appended_df = appended_df.dropna(subset=['Date'])

# Keep only specific columns
columns_to_keep = [
    'Type', 'Date', 'Num', 'Name', 'Memo', 'Item', 'Item Description',
    'Account', 'Class', 'Split', 'Paid', 'Sales Price', 'Debit', 'Credit',
    'Amount', 'Balance', 'Account Type', 'Amount Paid', 'Name Account #'
]
appended_df = appended_df[columns_to_keep]

# Clean up column names
appended_df.columns = appended_df.columns.str.strip().str.replace(' ', '_')





In [ ]:
# Save the cleaned and appended dataframe to a new Excel file
cleaned_file = 'DPR_GL_YTD_Appended_Cleaned.xlsx'
appended_df.to_excel(cleaned_file, index=False, engine='openpyxl')  # Specify engine for .xlsx files

print("Appended and cleaned data saved to", cleaned_file)

In [106]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font

# Read your data and preprocess if necessary
# ...

# Define account types related to income statement
income_statement_account_types = [
    'Income', 'Cost of Goods Sold', 'Expense', 'Other Expense'
]

# Filter data to include only income statement account types
income_statement_df = appended_df[appended_df['Account_Type'].isin(income_statement_account_types)]

# Create a new column for the month
income_statement_df['Month'] = income_statement_df['Date'].dt.to_period('M')

# Create a pivot table for the P&L report
pl_report = pd.pivot_table(income_statement_df, 
                           index='Account', 
                           columns='Month', 
                           values='Amount', 
                           aggfunc='sum', 
                           fill_value=0)

# Calculate the subtotal rows
pl_report['Subtotal'] = pl_report.sum(axis=1, numeric_only=True)

# Convert Period values to strings using a list comprehension
pl_report.columns = [col.strftime('%b-%y') if isinstance(col, pd.Period) else col for col in pl_report.columns]

# Convert values to thousands
pl_report = pl_report.apply(lambda x: x / 1000)

# Create a new Excel workbook
output_excel_file = 'standard_pl_report.xlsx'
workbook = Workbook()
worksheet = workbook.active

# Set header styles
header_font = Font(bold=True, color="000080")
header_alignment = Alignment(horizontal='center')
for col_idx, column in enumerate(pl_report.columns, start=2):
    header_cell = worksheet.cell(row=5, column=col_idx, value=column)
    header_cell.font = header_font
    header_cell.alignment = header_alignment

# Populate the P&L report data
for row_idx, (account, data) in enumerate(pl_report.iterrows(), start=6):
    worksheet.cell(row=row_idx, column=1, value=account)
    for col_idx, value in enumerate(data, start=2):
        worksheet.cell(row=row_idx, column=col_idx, value=value)

# Apply accounting format without decimals
accounting_format = '"$"#,##0'
for row in worksheet.iter_rows(min_row=6, max_row=worksheet.max_row, min_col=2, max_col=worksheet.max_column):
    for cell in row:
        cell.number_format = accounting_format

# Save the workbook
workbook.save(output_excel_file)

print(f"Standard P&L report exported to '{output_excel_file}'")


Standard P&L report exported to 'standard_pl_report.xlsx'


/var/folders/_h/thz7n6kj0d9bb5xy0x1wtcz40000gn/T/ipykernel_86527/73470789.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_statement_df['Month'] = income_statement_df['Date'].dt.to_period('M')


In [112]:
#Create Balance Sheet

# Define balance sheet account types
balance_sheet_account_types = [
    'Bank', 'Accounts Receivable', 'Other Current Asset', 'Fixed Asset',
    'Accounts Payable', 'Credit Card', 'Other Current Liability', 'Equity',
    'Other Asset'
]

# Filter data to include only balance sheet account types
balance_sheet_df = appended_df[appended_df['Account_Type'].isin(balance_sheet_account_types)]

# Create a new column for the month
balance_sheet_df['Month'] = balance_sheet_df['Date'].dt.to_period('M')

# Calculate the net balance for each account by month
balance_sheet_df['Net_Balance'] = balance_sheet_df.groupby(['Account', 'Month'])['Amount'].transform('sum')

# Pivot the data to create the balance sheet report
balance_sheet_report = pd.pivot_table(balance_sheet_df, 
                                      index='Account', 
                                      columns='Month', 
                                      values='Net_Balance', 
                                      aggfunc='sum', 
                                      fill_value=0)

# Convert Period values to strings using a list comprehension
balance_sheet_report.columns = [col.strftime('%b-%y') if isinstance(col, pd.Period) else col for col in balance_sheet_report.columns]

# Calculate the rolling balance by adding previous period's ending balance to the net change
for col_idx in range(1, balance_sheet_report.shape[1]):
    balance_sheet_report.iloc[:, col_idx] = balance_sheet_report.iloc[:, col_idx] + balance_sheet_report.iloc[:, col_idx - 1]

# Create a new Excel workbook
output_excel_file = 'rolling_balance_sheet_report.xlsx'
workbook = Workbook()
worksheet = workbook.active

# Set header styles
header_font = Font(bold=True, color="000080")
header_alignment = Alignment(horizontal='center')
for col_idx, column in enumerate(balance_sheet_report.columns, start=2):
    header_cell = worksheet.cell(row=5, column=col_idx, value=column)
    header_cell.font = header_font
    header_cell.alignment = header_alignment

# Populate the rolling balance sheet report data
for row_idx, (account, data) in enumerate(balance_sheet_report.iterrows(), start=6):
    worksheet.cell(row=row_idx, column=1, value=account)
    for col_idx, value in enumerate(data, start=2):
        worksheet.cell(row=row_idx, column=col_idx, value=value)

# Apply accounting format without decimals
accounting_format = '"$"#,##0'
for row in worksheet.iter_rows(min_row=6, max_row=worksheet.max_row, min_col=2, max_col=worksheet.max_column):
    for cell in row:
        cell.number_format = accounting_format

# Save the workbook
workbook.save(output_excel_file)

print(f"Rolling balance sheet report exported to '{output_excel_file}'")


Rolling balance sheet report exported to 'rolling_balance_sheet_report.xlsx'


/var/folders/_h/thz7n6kj0d9bb5xy0x1wtcz40000gn/T/ipykernel_86527/3316542523.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_df['Month'] = balance_sheet_df['Date'].dt.to_period('M')
/var/folders/_h/thz7n6kj0d9bb5xy0x1wtcz40000gn/T/ipykernel_86527/3316542523.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_df['Net_Balance'] = balance_sheet_df.groupby(['Account', 'Month'])['Amount'].transform('sum')


In [87]:
# Write the styled P&L report to the Excel workbook
pl_report_final_styled.to_excel(writer, startrow=5, startcol=1, na_rep='', header=False, index=True)

# Access the underlying openpyxl workbook and worksheet
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Add the blank column and rows
worksheet.insert_cols(1)
worksheet.insert_rows(1, amount=5)

# Set font for header rows
font = Font(bold=True, color="000080")
for cell in worksheet[5]:
    cell.font = font



In [88]:

# Add a new column for the month
appended_df['Month'] = appended_df['Date'].dt.to_period('M')

# Create a month-year format for the column headers
pl_report = appended_df.pivot_table(index='Account', columns='Month', values='Amount', aggfunc='sum', fill_value=0)
pl_report.columns = pl_report.columns.to_timestamp().strftime('%b-%y')

# Group by Account and Account Type, and sum the values
pl_report_grouped = pl_report.groupby('Account').sum(min_count=1)

# Calculate subtotals for each account type
account_types = [
    'Bank', 'Accounts Receivable', 'Other Current Asset', 'Fixed Asset',
    'Accounts Payable', 'Credit Card', 'Other Current Liability',
    'Equity', 'Income', 'Cost of Goods Sold', 'Expense',
    'Other Expense', 'Other Asset'
]

# Initialize an empty DataFrame to store the final report
pl_report_final = pd.DataFrame(columns=pl_report_grouped.columns)

# Iterate over the account types and add subtotals
for acc_type in account_types:
    acc_list = appended_df.loc[appended_df['Account_Type'] == acc_type, 'Account'].unique()
    acc_subtotal = pl_report_grouped.loc[acc_list].sum()
    pl_report_final.loc[acc_type] = acc_subtotal

# Styling the header
header_style = Font(bold=True, color="000080")
pl_report_final_styled = pl_report_final.style.set_table_styles([{'selector': 'th', 'props': header_style}])

# Formatting values in accounting format (000's and no decimals)
pl_report_final_styled = pl_report_final_styled.format('${:,.0f}')

# Create a new Excel workbook
output_excel_file = 'styled_pl_report_with_subtotals.xlsx'
workbook = Workbook()
writer = pd.ExcelWriter(output_excel_file, engine='openpyxl')
writer.book = workbook

# Write the styled P&L report to the Excel workbook
pl_report_final_styled.to_excel(writer, startrow=5, startcol=1, na_rep='', float_format='%.0f', header=False, index=True)

# Access the openpyxl worksheet object
worksheet = writer.sheets['Sheet1']

# Set the row height for the subtotal rows to 3 times the default row height (15)
for idx in range(6, len(account_types) + 6):
    worksheet.row_dimensions[idx].height = 3 * 15

# Save the workbook
writer.save()

print(f"Styled P&L report with subtotals exported to '{output_excel_file}'")




AttributeError: can't set attribute

In [50]:

# Export the styled DataFrame to an Excel file
output_excel_file = 'styled_pl_report.xlsx'
pl_report_styled.to_excel(output_excel_file, engine='openpyxl', na_rep='')

print(f"Styled P&L report exported to '{output_excel_file}'")

Styled P&L report exported to 'styled_pl_report.xlsx'


In [42]:
# Call the analysis function
pl_report_df = generate_pl_report(appended_df)

# Print the generated P&L report
print(pl_report_df)

Month                                    2021-01    2021-02    2021-03  \
Account                                                                  
40000 · Client Fees                         0.00       0.00       0.00   
40010 · Detox Revenue                 -169836.00  -88060.00 -149507.00   
40020 · Residential Revenue           -455707.00 -306270.00 -393492.00   
40030 · PHP Revenue                   -125770.00 -237021.00 -261092.00   
40040 · IOP Revenue                    -56903.00  -69008.00  -71063.00   
40050 · OP Revenue                      -1709.00   -5099.00   -2796.00   
40070 · Ancilliary Services Revenue       -92.00   -1698.00   -1454.00   
40080 · Private Pay Revenue                 0.00  -33175.00  -98500.00   
40600 · Patient Refunds                  7061.10       0.00     793.65   
60200 · Auto                                0.00       0.00       0.00   
60210 · Gasoline Petrol                  1142.21    1015.29    1719.79   
60220 · Insurance Registration        

In [43]:
# Specify the output Excel file name
output_file = 'profit_and_loss_report.xlsx'

# Export the P&L report to an Excel file
formatted_pl_report.to_excel(output_file, sheet_name='Profit and Loss Report')
print(f'Profit and Loss Report exported to: {output_file}')

Profit and Loss Report exported to: profit_and_loss_report.xlsx


In [63]:
print(appended_df['Account_Type'].unique())

['Bank' 'Accounts Receivable' 'Other Current Asset' 'Fixed Asset'
 'Accounts Payable' 'Credit Card' 'Other Current Liability' 'Equity'
 'Income' 'Cost of Goods Sold' 'Expense' 'Other Expense' 'Other Asset']


In [ ]:

# Call the analysis functions
income_statement_df = generate_income_statement(appended_df)
# Save the analysis results to separate tabs in the Excel file
with pd.ExcelWriter(cleaned_file, engine='openpyxl') as writer:
    appended_df.to_excel(writer, sheet_name='Appended Data', index=False)
    income_statement_df.to_excel(writer, sheet_name='Income Statement', index=False)
    
    print("Appended, cleaned, and analyzed data saved to", cleaned_file)

In [ ]:
# Example of integrating balance sheet analysis
def generate_balance_sheet(dataframe):
    # Your logic to generate balance sheet
    return balance_sheet_df

# Example of integrating AR aging analysis
def generate_ar_aging(dataframe):
    # Your logic to generate AR aging
    return ar_aging_df

# Example of integrating AP aging analysis
def generate_ap_aging(dataframe):
    # Your logic to generate AP aging
    return ap_aging_df

# Example of integrating net working capital analysis
def generate_working_capital(dataframe):
    # Your logic to generate working capital analysis
    return working_capital_df


In [ ]:


# Example of integrating balance sheet analysis
def generate_balance_sheet(dataframe):
    # Your logic to generate balance sheet
    return balance_sheet_df

# Example of integrating AR aging analysis
def generate_ar_aging(dataframe):
    # Your logic to generate AR aging
    return ar_aging_df

# Example of integrating AP aging analysis
def generate_ap_aging(dataframe):
    # Your logic to generate AP aging
    return ap_aging_df

# Example of integrating net working capital analysis
def generate_working_capital(dataframe):
    # Your logic to generate working capital analysis
    return working_capital_df

# Call the analysis functions
income_statement_df = generate_income_statement(appended_df)
balance_sheet_df = generate_balance_sheet(appended_df)
ar_aging_df = generate_ar_aging(appended_df)
ap_aging_df = generate_ap_aging(appended_df)
working_capital_df = generate_working_capital(appended_df)

# Save the analysis results to separate tabs in the Excel file
with pd.ExcelWriter(cleaned_file, engine='openpyxl') as writer:
    appended_df.to_excel(writer, sheet_name='Appended Data', index=False)
    income_statement_df.to_excel(writer, sheet_name='Income Statement', index=False)
    balance_sheet_df.to_excel(writer, sheet_name='Balance Sheet', index=False)
    ar_aging_df.to_excel(writer, sheet_name='AR Aging', index=False)
    ap_aging_df.to_excel(writer, sheet_name='AP Aging', index=False)
    working_capital_df.to_excel(writer, sheet_name='Working Capital', index=False)

print("Appended, cleaned, and analyzed data saved to", cleaned_file)



In [ ]:
# Step 3: Categorization based on Account Types
def categorize_account_type(account_type):
    if account_type in ['Bank', 'Accounts Receivable', 'Other Current Asset']:
        return 'Current Asset'
    elif account_type == 'Fixed Asset':
        return 'Fixed Asset'
    elif account_type in ['Accounts Payable', 'Credit Card', 'Other Current Liability']:
        return 'Current Liability'
    elif account_type in ['Equity']:
        return 'Equity'
    elif account_type in ['Income']:
        return 'Income'
    elif account_type in ['Cost of Goods Sold', 'Expense', 'Other Expense']:
        return 'Expense'
    else:
        return 'Other'

gl_data['Account_Segment'] = gl_data['Account Type'].apply(categorize_account_type)

In [ ]:
# Step 4: Calculate Financial Metrics
gl_data['Net_Income'] = gl_data.apply(lambda row: row['Amount'] if row['Account_Segment'] == 'Income' else -row['Amount'] if row['Account_Segment'] == 'Expense' else 0, axis=1)

# Step 5: Create Summary Report and Export to Excel
summary_report = {
    'Total Transactions': int(gl_data.shape[0]),
    'Total Accounts': int(gl_data['Account'].nunique()),
    'Total Amount': int(gl_data['Amount'].sum()),
    'Net Income': int(gl_data['Net_Income'].sum()),
    # Add more summary metrics as needed...
}

In [ ]:
# Convert the summary report to a DataFrame
summary_df = pd.DataFrame(summary_report.items(), columns=['Metric', 'Value'])


In [ ]:
print(summary_df)

In [ ]:
# Print unique account type names in separate lines
unique_account_types = gl_data['Account Type'].unique()
print("Unique Account Types:")
for account_type in unique_account_types:
    print(account_type)


In [ ]:
print("Summary exported to:", output_file)

In [ ]:
gldata_df=pd.DataFrame(gl_data)

In [ ]:
gldata_df.to_excel(os.path.join(desired_working_dir, 'GL Data.xlsx'), index=False)

In [ ]:
#Print Net Income
print(int(gl_data['Net_Income'].sum()))

In [ ]:
# Export the summary to an Excel workbook
output_file = 'financial_due_diligence_summary.xlsx'
with pd.ExcelWriter(output_file) as writer:
    gl_data.to_excel(writer, sheet_name='Raw Data', index=False)
    summary_df.to_excel(writer, sheet_name='Summary', index=False)

print("Summary exported to:", output_file)